In [1]:
import pandas as pd
from fbprophet import Prophet
from etl_resources import sqlite_connection
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from pandas_profiling import ProfileReport
plt.style.use('fivethirtyeight')
import pandasql as psql
pd.options.mode.chained_assignment = None 

In [1]:
from keras.models import Sequential
from keras.layers import Dense, LSTM

2021-11-28 18:26:31.443044: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-28 18:26:31.443079: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def ticker_list():
    
    con = sqlite_connection()
    cur = con.cursor()
    cur.execute('''
    select distinct w.ticker from weekly_prices_clean w
        inner join (select ticker from weekly_prices_clean
        group by ticker
        having max(date) > '2021-01-01') t on t.ticker=w.ticker
    ''')
    res = cur.fetchall()
    res = [val[0] for val in res]
    
    return res

In [3]:
def base_df(ticker):
    
    '''
    This function returns the base time series dataframe (date and close)
    '''
    
    con = sqlite_connection()
        
    df = pd.read_sql(f'''select date,close 
    from weekly_prices_clean where ticker='{ticker}' and date>'2017-12-31' 
    group by date,close
    order by date asc''',con=con)
    
    df[['ds', 'y']] = df[['date','close']]
    df = df[['ds', 'y']]
        
    return df

In [4]:
def train_test_split(df, split=0.2):
    
    test_rows = int(round(df.shape[0] * split,0))
    train_rows = df.shape[0] - test_rows
    
    test_df = df.head(test_rows).copy()
    train_df = df.tail(train_rows).copy()
    
    return train_df, test_df

In [ ]:
def build_lstm(x_train, y_train, x_test, y_test):
    
    
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape= (x_train.shape[1], 1)))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(1))

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    model.fit(x_train, y_train, batch_size=1, epochs=1)
    
    # Convert the data to a numpy array
    x_test = np.array(x_test)

    # Reshape the data
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1 ))

    # Get the models predicted price values 
    predictions = model.predict(x_test)
    predictions = scaler.inverse_transform(predictions)

    # Get the root mean squared error (RMSE)
    rmse = np.sqrt(np.mean(((predictions - y_test) ** 2)))
    #rmse
    

In [5]:
def mean_abs_perc_err(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [6]:
def percentage_price_change(base_df, fcast):
    
    qry = '''select y from base_df where ds in (select max(ds) from base_df)'''
    
    last_price = psql.sqldf(qry, locals())
    last_price = last_price.iloc[0]['y']
    
    
    three_mth_fcast = fcast[fcast['ds'] == '2022-02-04'] 
    three_mth_fcast['interval'] = three_mth_fcast['yhat_upper'] - three_mth_fcast['yhat_lower']
    three_mth_fcast = three_mth_fcast[['yhat','interval']]
    thr_mth_yhat = three_mth_fcast.iloc[0]['yhat']
    thr_mth_int = three_mth_fcast.iloc[0]['interval']
    thr_mth_int_perc = round(thr_mth_int/thr_mth_yhat,4)
    
    perc_chg = round((thr_mth_yhat - last_price)/last_price, 4)
    print('Last price:', last_price)
    print('3 month yhat:', thr_mth_yhat)
    print('Interval', thr_mth_int)
    print('% interval:', thr_mth_int_perc)
    print('% change:', perc_chg)
    
    return last_price, thr_mth_yhat, thr_mth_int, thr_mth_int_perc, perc_chg

In [7]:
def train_prophet():
    
    tickers = ticker_list()
    metrics_list = list()
    
    for ticker in tickers:
        
        print(f"Building model for {ticker}")
        
        metrics = dict()
        
        # Build the dataset and split
        df = base_df(ticker)
        train, test = train_test_split(df, split=0.15)
        
        # Train & Test Prophet
        model = Prophet(daily_seasonality=True)
        model.fit(train)
        forecast = model.predict(test)
        
        # Capture Metrics
        metrics['ticker'] = ticker
        metrics['MSE'] = mean_squared_error(y_true = test["y"], y_pred = forecast['yhat'])
        metrics['MAE'] = mean_absolute_error(y_true = test["y"], y_pred = forecast['yhat'])
        metrics['MAPE'] = mean_abs_perc_err(y_true = np.asarray(test["y"]), y_pred = np.asarray(forecast['yhat']))
        # TODO: Look into Symmetric MAPE 
        
        # Create a DF with future dates to predict (weekly candence)
        future = pd.DataFrame()
        future['ds'] = pd.date_range('2018-01-05', periods=250, freq='W-FRI')

        # Make the predictions
        future_fcast = model.predict(future)
        
        last_price, thr_mth_yhat, thr_mth_int, thr_mth_int_perc, perc_chg = percentage_price_change(df, future_fcast)
        
        metrics['last_price'] = last_price
        metrics['3_month_yhat'] = thr_mth_yhat
        metrics['interval'] = thr_mth_int
        metrics['interval_perc'] = thr_mth_int_perc
        metrics['3_month_perc_change'] = perc_chg
        metrics_list.append(metrics)
        # Plot and Save
        model.plot(future_fcast).savefig(f'../data/visualization/prophet/{ticker}.png')
        
    
    total_metrics = pd.DataFrame(metrics_list)
    
    # TODO: Audit this output
    total_metrics.to_csv('forecast_metrics.csv')
    
    return total_metrics

In [8]:
train_prophet()

Building model for MMM
Initial log joint probability = -3.13007
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       421.435     0.0104366       125.513           1           1      130   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     146       423.325   0.000924563       257.837   5.985e-06       0.001      230  LS failed, Hessian reset 
     194       424.272   0.000294251       109.663   4.762e-06       0.001      328  LS failed, Hessian reset 
     199       424.321   0.000378461       48.1601       2.062      0.5737      335   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     248       424.478   0.000174195       84.0378   2.497e-06       0.001      435  LS failed, Hessian reset 
     299       424.506   0.000372868       88.5079          10           1      505   
    Iter      log prob        ||dx||      ||grad||       alpha    

/home/james-laptop/Desktop/RPI_Projects/rensselaer_ai_systems/venv/lib/python3.8/site-packages/fbprophet/plot.py:66: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(facecolor='w', figsize=figsize)


Initial log joint probability = -2.30591
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      88       509.463    0.00020946       140.848   2.698e-06       0.001      146  LS failed, Hessian reset 
      99       509.532   6.55318e-05       70.3534           1           1      161   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     144       509.575   0.000148173       115.471   1.413e-06       0.001      250  LS failed, Hessian reset 
     199       509.597   1.71674e-06       71.0075           1           1      322   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     236        509.63   0.000132407       97.2406    1.62e-06       0.001      404  LS failed, Hessian reset 
     299       509.646   5.29434e-05       100.747           1           1      479   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Note

,ticker,MSE,MAE,MAPE,last_price,3_month_yhat,interval,interval_perc,3_month_perc_change
0,MMM,252.485590,13.973927,6.614179,180.61,214.810257,24.913836,0.1160,0.1894
1,BKNG,66468.851374,234.770426,11.326041,2437.01,2674.096511,390.544737,0.1460,0.0973
2,ABT,22.841405,3.868956,6.320991,129.09,130.568925,8.902181,0.0682,0.0115
3,ABBV,80.236632,7.803183,7.801892,117.06,125.359557,10.727018,0.0856,0.0709
4,ACN,731.124992,25.244993,15.955007,366.80,365.360757,24.481978,0.0670,-0.0039
...,...,...,...,...,...,...,...,...,...
92,WBA,263.797069,15.806811,23.521413,49.51,63.434488,10.778716,0.1699,0.2812
93,DIS,50.477726,5.851010,5.664372,170.28,214.935683,34.439285,0.1602,0.2622
94,WFC,7.874566,2.186839,3.868527,50.62,60.584755,11.428675,0.1886,0.1969
95,HON,32.195302,4.197833,2.734957,222.49,258.177252,35.359373,0.1370,0.1604


In [9]:
def quant_calc(row, quant_dict):
        
    if row <= quant_dict.get(.25):
        return 1
    elif row > quant_dict.get(.25) and row <= quant_dict.get(.5):
        return 2
    elif row > quant_dict.get(.5) and row <= quant_dict.get(.75):
        return 3
    else:
        return 4

In [13]:
def quantiles_score(df):
    
    '''
    Quartile Labeling
    '''
    
    cols = ['MAPE','interval_perc','3_month_perc_change']
    
    for col in cols:
        
        quant_dict = {
        .25 : df[col].quantile(.25),
        .5 : df[col].quantile(.5),
        .75 : df[col].quantile(.75)
        }

        df[col+'_Score'] = df[col].apply(lambda row: quant_calc(row, quant_dict))
    
    df['MAPE_Score'] = df['MAPE_Score'] * -1
    df['interval_perc_Score'] = df['interval_perc_Score'] * -1
    
    df['Strategy_Score'] = df['MAPE_Score'] + df['3_month_perc_change_Score']
    
    return df

In [14]:
def strategy():
    
    metrics_df = pd.read_csv('forecast_metrics.csv')
    
    profile = ProfileReport(metrics_df, title='Prophet Metrics')
    profile.to_file('../data/profiles/prophet_metrics.html')
    
    metrics_df = quantiles_score(metrics_df)
    
    metrics_df.to_csv('strategy.csv')

In [15]:
strategy()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]